In [1]:
from pyspark import SparkContext, SparkConf # 导入相关工具包

# 初始化Spark上下文
# local为本地调试模式，具体集群方式参照http://spark.apache.org/docs/latest/cluster-overview.html
conf = SparkConf().setAppName("CF").setMaster("local") 
sc = SparkContext(conf=conf)
print ("init complete：sc = ", sc)

('init complete\xef\xbc\x9asc = ', <SparkContext master=local appName=CF>)


In [7]:
# 导入数据
data = sc.textFile("file:///root/notebook/data/ratings.dat").map(lambda line: line.split("::"))
print("data.count() = %d" % data.count())


data.count() = 10000


In [3]:
print data.collect()

[[u'1', u'122', u'5', u'838985046'], [u'1', u'185', u'5', u'838983525'], [u'1', u'231', u'5', u'838983392'], [u'1', u'292', u'5', u'838983421'], [u'1', u'316', u'5', u'838983392'], [u'1', u'329', u'5', u'838983392'], [u'1', u'355', u'5', u'838984474'], [u'1', u'356', u'5', u'838983653'], [u'1', u'362', u'5', u'838984885'], [u'1', u'364', u'5', u'838983707'], [u'1', u'370', u'5', u'838984596'], [u'1', u'377', u'5', u'838983834'], [u'1', u'420', u'5', u'838983834'], [u'1', u'466', u'5', u'838984679'], [u'1', u'480', u'5', u'838983653'], [u'1', u'520', u'5', u'838984679'], [u'1', u'539', u'5', u'838984068'], [u'1', u'586', u'5', u'838984068'], [u'1', u'588', u'5', u'838983339'], [u'1', u'589', u'5', u'838983778'], [u'1', u'594', u'5', u'838984679'], [u'1', u'616', u'5', u'838984941'], [u'2', u'110', u'5', u'868245777'], [u'2', u'151', u'3', u'868246450'], [u'2', u'260', u'5', u'868244562'], [u'2', u'376', u'3', u'868245920'], [u'2', u'539', u'3', u'868246262'], [u'2', u'590', u'5', u'8682